# Introduction

This notebook uses the NAMCS for the years 2005-2016 in SPSS (.sav files) from https://www.cdc.gov/nchs/ahcd/datasets_documentation_related.htm. More SPSS data is available from years 2002-2016, and STATA (2000-2016; .do, .dct, .exe files). 

Google Health Searches data is available for years 2004-2017, so the goal here is to see how Google Health Searches correlate with prevalence/ proportion (?) of diseases including Cardiovascular Disearse, Stroke, Cancer, Diabetes, Obesity, and Depression in the years 2005-2016. 2004 data ommitted for this project due to large differences in survey labels.

In [0]:
import pandas as pd
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import requests
!pip install pyreadstat
import pyreadstat
from collections import Counter
import re
import time


     |████████████████████████████████| 1.9MB 2.8MB/s 


In [0]:
# Mount Google Drive files to Colab
# SPSS Data files (.sav format) were uploaded to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Read SPSS file

In [0]:
# Create list of all years and filenames for the CDC data
years = []
filenames = []

for i in range(5,17):
    if i < 10:
        yr = '200' + str(i)
        temp = 'namcs0' + str(i) + '-spss.sav'
    else:
        yr = '20' + str(i)
        temp = 'namcs' + yr + '-spss.sav'    

    years.append(yr)
    filenames.append(temp)

print(years) 
print(filenames)

['2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']
['namcs05-spss.sav', 'namcs06-spss.sav', 'namcs07-spss.sav', 'namcs08-spss.sav', 'namcs09-spss.sav', 'namcs2010-spss.sav', 'namcs2011-spss.sav', 'namcs2012-spss.sav', 'namcs2013-spss.sav', 'namcs2014-spss.sav', 'namcs2015-spss.sav', 'namcs2016-spss.sav']


In [0]:
# This study will use 12 years of data from 2005 - 2016

len(filenames)

12

# Data Cleaning / Wrangling

In [0]:
# Initialize dataframe and dictionary with total counts
trim_df = pd.DataFrame()
all_years_counts = dict()
start = time.time()

# Loop to concatenate (vertically stack) SPSS data files for all years
for ind in range(len(filenames)):
  ### Read in the data
  file = filenames[ind]
  path = '/content/drive/My Drive/Capstone1/' + file
  #file='/content/drive/My Drive/Capstone1/namcs2016-spss.sav' 
  df, meta = pyreadstat.read_sav(path, apply_value_formats=True, formats_as_category=True)
 
  # Extract the year from filename
  temp = re.findall(r'\d{2,4}', file) # find all digits in file name that are 2-4 digits long
  if len(temp[0]) == 4:
      year = int(temp[0])
  else:
      year = int('20' + temp[0])

  print('Completed reading in data from the year', year)
  print('The shape of the dataframe is', df.shape)
  
  # Create a column for year
  df['YEAR'] = year
  
  # Create subset, extract only the columns that are relevant to project
  if year < 2012:
    # For 2005-2011 datasets
    colnames_sub =  ['YEAR', 'VMONTH', 'AGE', 'AGER', 'SEX', 'PATWT',
                    'REGION', 'RETYPOFF', 'MSA',
                    'RFV1', 'RFV2', 'RFV3', 'PRIMCARE', 'REFER', 
                    'MAJOR', 'DIAG1', 'DIAG2', 'DIAG3',
                    'TIMEMD', 'REFOTHMD', 
                    'CANCER', 'DIABETES', 'IHD', 'CHF', 'CEBVD',
                    'DEPRN', 'OBESITY']
    subset = df[colnames_sub]
    # Replace 'IHD' with 'CAD' column name
    subset.rename(columns = {'IHD':'CAD'}, inplace=True)
    
  elif year == 2012 or year == 2013:
    # For 2012-2013 datasets
    colnames_sub =  ['YEAR', 'VMONTH', 'AGE', 'AGER', 'SEX', 'PATWT',
                    'REGIONOFF', 'RETYPOFFR', 'MSA',
                    'RFV1', 'RFV2', 'RFV3', 'PRIMCARE', 'REFER', 
                    'MAJOR', 'DIAG1', 'DIAG2', 'DIAG3',
                    'TIMEMD', 'REFOTHMD', 
                    'CANCER', 'DIABETES', 'IHD', 'CHF', 'CEBVD', 
                    'DEPRN', 'OBESITY']
    subset = df[colnames_sub]
    # Replace 'IHD' with 'CAD' column name, 'REGIONOFF' with 'REGION', etc.
    subset.rename(columns = {'IHD':'CAD', 'REGIONOFF':'REGION', 'RETYPOFFR':'RETYPOFF'}, inplace=True)

  else:
    # For 2014-2016 datasets
    colnames_sub =  ['YEAR', 'VMONTH', 'AGE', 'AGER', 'SEX', 'PATWT',
                    'REGIONOFF', 'RETYPOFFR', 'MSA',
                    'RFV1', 'RFV2', 'RFV3', 'PRIMCARE', 'REFER', 
                    'MAJOR', 'DIAG1', 'DIAG2', 'DIAG3',
                    'TIMEMD', 'REFOTHMD', 
                    'CANCER', 'DIABTYP1', 'DIABTYP2', 'DIABTYP0',
                    'CAD', 'CHF', 'CEBVD',
                    'DEPRN', 'OBESITY']
    subset = df[colnames_sub]
    subset.rename(columns = {'REGIONOFF':'REGION', 'RETYPOFFR':'RETYPOFF'}, inplace=True)
    bool1 = subset['DIABTYP1'] == 'Yes'
    bool2 = subset['DIABTYP2'] == 'Yes'
    bool3 = subset['DIABTYP0'] == 'Yes'
    db = subset[bool1 | bool2 | bool3]
    # Create new column 'DIABETES' to consolidate 'DIABTYP1', 'DIABTYP2', 'DIABTYP0'
    subset['DIABETES'] = 'No' # default column value
    # unless patient has Diabetes type 1, 2, or 'unspecified' Diabetes
    for i in db.index:
      subset.loc[i,'DIABETES'] = 'Yes'
    # Remove 'DIABTYP1', 'DIABTYP2', 'DIABTYP0' columns
    subset = subset.drop(['DIABTYP1', 'DIABTYP2', 'DIABTYP0'], axis=1)
    # Reorder columns
    subset = subset[['YEAR', 'VMONTH', 'AGE', 'AGER', 'SEX', 'PATWT', 'REGION',
       'RETYPOFF', 'MSA', 'RFV1', 'RFV2', 'RFV3', 'PRIMCARE', 'REFER',
       'MAJOR', 'DIAG1', 'DIAG2', 'DIAG3', 'TIMEMD', 'CANCER', 'DIABETES',
       'CAD', 'CHF', 'CEBVD', 'DEPRN', 'OBESITY']]

  #Count the occurences of each disease, and build a dictionary for each year
  #if year < 2014:
    # For 2005-2013 datasets, list of disease names
  diags = ['CANCER', 'DIABETES', 'CAD', 'CHF', 'CEBVD', 'DEPRN', 'OBESITY']
  d0 = subset[diags[0]]=='Yes'
  d1 = subset[diags[1]]=='Yes'
  d2 = subset[diags[2]]=='Yes'
  d3 = subset[diags[3]]=='Yes'
  d4 = subset[diags[4]]=='Yes'
  d5 = subset[diags[5]]=='Yes'
  d6 = subset[diags[6]]=='Yes'
  temp_df = subset[d0 | d1 | d2 | d3 | d4 | d5 | d6]
  trim_df = pd.concat([trim_df, temp_df], axis=0, ignore_index=True, sort=False)
  
  n_all = dict() # initialize dictionary to count diseases for current year in loop
  for i in diags:
    n_yes = len(subset[subset[i] == 'Yes'])
    n_all[i] = n_yes
    
  # Add dictionary to large dictionary with all years
  all_years_counts[year] = n_all
  tcheck = time.time()
  print('Time to complete loop', ind, ':', tcheck-start)

print(all_years_counts)
print(trim_df.head())

Completed reading in data from the year 2005
The shape of the dataframe is (25665, 354)
Time to complete loop 

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(**kwargs)


0 : 11.589228391647339
Completed reading in data from the year 2006
The shape of the dataframe is (29392, 449)
Time to complete loop 1 : 30.226481676101685
Completed reading in data from the year 2007
The shape of the dataframe is (32778, 453)
Time to complete loop 2 : 52.9420280456543
Completed reading in data from the year 2008
The shape of the dataframe is (28741, 453)
Time to complete loop 3 : 72.48630857467651
Completed reading in data from the year 2009
The shape of the dataframe is (32281, 440)
Time to complete loop 4 : 93.87728476524353
Completed reading in data from the year 2010
The shape of the dataframe is (31229, 481)
Time to complete loop 5 : 116.93627572059631
Completed reading in data from the year 2011
The shape of the dataframe is (30872, 480)
Time to complete loop 6 : 139.44848155975342
Completed reading in data from the year 2012
The shape of the dataframe is (76330, 624)
Time to complete loop 7 : 222.81648993492126
Completed reading in data from the year 2013
The s

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Time to complete loop 9 : 411.74905228614807
Completed reading in data from the year 2015
The shape of the dataframe is (28332, 1096)
Time to complete loop 10 : 487.84825706481934
Completed reading in data from the year 2016
The shape of the dataframe is (13165, 1031)
Time to complete loop 11 : 519.5233306884766
{2005: {'CANCER': 1619, 'DIABETES': 2266, 'CAD': 1233, 'CHF': 431, 'CEBVD': 574, 'DEPRN': 2704, 'OBESITY': 1651}, 2006: {'CANCER': 2270, 'DIABETES': 2882, 'CAD': 1403, 'CHF': 499, 'CEBVD': 530, 'DEPRN': 2858, 'OBESITY': 1986}, 2007: {'CANCER': 2996, 'DIABETES': 3438, 'CAD': 1638, 'CHF': 601, 'CEBVD': 631, 'DEPRN': 3102, 'OBESITY': 2010}, 2008: {'CANCER': 1645, 'DIABETES': 2955, 'CAD': 1318, 'CHF': 462, 'CEBVD': 562, 'DEPRN': 2727, 'OBESITY': 1813}, 2009: {'CANCER': 1892, 'DIABETES': 3873, 'CAD': 1639, 'CHF': 668, 'CEBVD': 662, 'DEPRN': 3458, 'OBESITY': 2576}, 2010: {'CANCER': 2661, 'DIABETES': 3434, 'CAD': 1314, 'CHF': 542, 'CEBVD': 526, 'DEPRN': 3130, 'OBESITY': 2321}, 2011: {

In [0]:
print(trim_df.info())
trim_df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142286 entries, 0 to 142285
Data columns (total 27 columns):
YEAR        142286 non-null int64
VMONTH      142286 non-null category
AGE         142286 non-null object
AGER        142286 non-null category
SEX         142286 non-null category
PATWT       142286 non-null float64
REGION      142286 non-null category
RETYPOFF    142286 non-null object
MSA         142286 non-null category
RFV1        142286 non-null object
RFV2        142286 non-null object
RFV3        142286 non-null object
PRIMCARE    142286 non-null category
REFER       142286 non-null category
MAJOR       142286 non-null object
DIAG1       142286 non-null object
DIAG2       142286 non-null object
DIAG3       142286 non-null object
TIMEMD      142286 non-null object
REFOTHMD    111930 non-null object
CANCER      142286 non-null category
DIABETES    142286 non-null object
CAD         142286 non-null category
CHF         142286 non-null category
CEBVD       142286 non-null c

,YEAR,VMONTH,AGE,AGER,SEX,PATWT,REGION,RETYPOFF,MSA,RFV1,RFV2,RFV3,PRIMCARE,REFER,MAJOR,DIAG1,DIAG2,DIAG3,TIMEMD,REFOTHMD,CANCER,DIABETES,CAD,CHF,CEBVD,DEPRN,OBESITY
142281,2016,January,69,65-74 years,Male,76524.39558,West,Other,MSA (Metropolitan Statistical Area),"Progress visit, NOS","Pain, unspecified","Tiredness, exhaustion",No,No,"Chronic problem, routine",Malignant neoplasm of prostate,Secondary malignant neoplasm of bone and bone ...,"Pain, not elsewhere classified",25 minutes,NaN,Yes,No,No,No,No,No,No
142282,2016,January,66,65-74 years,Male,76524.39558,West,Other,MSA (Metropolitan Statistical Area),"Pain, unspecified",Nausea,Blank,No,No,"Chronic problem, routine",Multiple myeloma,"Pain, not elsewhere classified",Blank,25 minutes,NaN,Yes,No,No,No,No,No,No
142283,2016,January,80,75 years and over,Female,76524.39558,West,Other,MSA (Metropolitan Statistical Area),"Medical counseling, NOS",Blood in stool (melena),Anal-rectal swelling or mass,No,Yes,"Chronic problem, flare-up","Malignant neoplasm of anus, unspecified",Type 2 diabetes mellitus without complications,Blank,1 hour,NaN,Yes,Yes,No,No,No,No,No
142284,2016,January,76,75 years and over,Female,76524.39558,West,Other,MSA (Metropolitan Statistical Area),Other symptoms referable to the nervo...,Anxiety and nervousness,Restlessness,No,No,"Chronic problem, routine",Malignant neoplasm of unspecified ovary,"Disorientation, unspecified",Restlessness and agitation,25 minutes,NaN,Yes,No,No,No,No,Yes,No
142285,2016,January,73,65-74 years,Male,76524.39558,West,Other,MSA (Metropolitan Statistical Area),"Progress visit, NOS",Blank,Blank,No,No,"Chronic problem, routine",Personal history of malignant neoplasm of urin...,Blank,Blank,15 minutes,NaN,Yes,No,No,No,No,No,No


# Saving Data

In [0]:
### Save the trim_df as csv and download

#trim_df.to_csv('cdc_namcs_trimmed.csv') 

#from google.colab import files

#files.download('cdc_namcs_trimmed.csv')

In [0]:
### all_years_counts dictionary
print(all_years_counts)

{2005: {'CANCER': 1619, 'DIABETES': 2266, 'CAD': 1233, 'CHF': 431, 'CEBVD': 574, 'DEPRN': 2704, 'OBESITY': 1651}, 2006: {'CANCER': 2270, 'DIABETES': 2882, 'CAD': 1403, 'CHF': 499, 'CEBVD': 530, 'DEPRN': 2858, 'OBESITY': 1986}, 2007: {'CANCER': 2996, 'DIABETES': 3438, 'CAD': 1638, 'CHF': 601, 'CEBVD': 631, 'DEPRN': 3102, 'OBESITY': 2010}, 2008: {'CANCER': 1645, 'DIABETES': 2955, 'CAD': 1318, 'CHF': 462, 'CEBVD': 562, 'DEPRN': 2727, 'OBESITY': 1813}, 2009: {'CANCER': 1892, 'DIABETES': 3873, 'CAD': 1639, 'CHF': 668, 'CEBVD': 662, 'DEPRN': 3458, 'OBESITY': 2576}, 2010: {'CANCER': 2661, 'DIABETES': 3434, 'CAD': 1314, 'CHF': 542, 'CEBVD': 526, 'DEPRN': 3130, 'OBESITY': 2321}, 2011: {'CANCER': 2699, 'DIABETES': 3649, 'CAD': 1376, 'CHF': 633, 'CEBVD': 743, 'DEPRN': 3815, 'OBESITY': 2430}, 2012: {'CANCER': 5479, 'DIABETES': 8138, 'CAD': 2461, 'CHF': 1219, 'CEBVD': 1420, 'DEPRN': 7124, 'OBESITY': 4639}, 2013: {'CANCER': 3750, 'DIABETES': 6399, 'CAD': 1968, 'CHF': 913, 'CEBVD': 1000, 'DEPRN': 541

In [0]:
### Convert all_year_counts dictionary to DataFrame
count_df = pd.DataFrame.from_dict(all_years_counts, orient='index')

### Convert DataFrame to CSV                      
#count_df.to_csv('all_counts.csv')

### Download file
from google.colab import files
files.download('all_counts.csv')

# Other / Testing

In [0]:
''' 
### Ended up accessing data files that were uploaded to Google Drive, instead of local computer files
# Upload data file from local computer here
from google.colab import files

start = time.time()

uploaded2 = files.upload()

for fn in uploaded2.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded2[fn])))
  
end = time.time()
print('The time to load file was', end - start)
'''

Saving namcs2016-spss.sav to namcs2016-spss.sav
User uploaded file "namcs2016-spss.sav" with length 23468228 bytes
The time to load file was 204.70199275016785


In [0]:
# Meta data from the SPSS file
'''
print(meta.column_names)
print(meta.column_labels)
print(meta.variable_to_label)
print(meta.value_labels)
'''

['VMONTH', 'VDAYR', 'AGE', 'AGER', 'AGEDAYS', 'SEX', 'PREGNANT', 'GESTWK', 'ETHUN', 'RACEUN', 'ETHIM', 'RACER', 'RACERETH', 'NOPAY', 'PAYPRIV', 'PAYMCARE', 'PAYMCAID', 'PAYWKCMP', 'PAYSELF', 'PAYNOCHG', 'PAYOTH', 'PAYDK', 'PAYTYPER', 'USETOBAC', 'INJURY', 'INJR1', 'INJR2', 'INJPOISAD', 'INJPOISADR1', 'INJPOISADR2', 'INTENT', 'INJDET_TRD', 'INJDETR1_TRD', 'INJDETR2_TRD', 'RFV1', 'RFV2', 'RFV3', 'RFV13D', 'RFV23D', 'RFV33D', 'PRIMCARE', 'REFER', 'SENBEFOR', 'PASTVIS', 'MAJOR', 'DIAG1', 'PRDIAG1', 'DIAG2', 'PRDIAG2', 'DIAG3', 'PRDIAG3', 'DIAG13D', 'DIAG23D', 'DIAG33D', 'DIAG1R', 'DIAG2R', 'DIAG3R', 'ARTHRTIS', 'ASTHMA', 'ASTH_SEV', 'ASTH_CON', 'CANCER', 'CASTAGE', 'CEBVD', 'COPD', 'CRF', 'CHF', 'DEPRN', 'DIABETES', 'HYPLIPID', 'HTN', 'IHD', 'OBESITY', 'OSTPRSIS', 'NOCHRON', 'TOTCHRON', 'HTTAKE', 'HTIN', 'WTTAKE', 'WTLB', 'BMI', 'TEMPTAKE', 'TEMPF', 'BLODPRES', 'BPSYS', 'BPDIAS', 'SERVICES', 'BREAST', 'DEPRESS', 'FOOT', 'PHYSICAL', 'NEURO', 'PELVIC', 'RECTAL', 'RETINAL', 'SKIN', 'CBC', 'GL

In [0]:
# More meta data from the SPSS file
print(meta.number_rows)
print(meta.number_columns)
print(meta.file_label)
print(meta.file_encoding)

76330
624
None
UTF-8


In [0]:
#print(meta.variable_value_labels)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
print(Counter(trim_df['REGION']))
print(Counter(trim_df['MSA']))
print(Counter(trim_df['RETYPOFF']))

Counter({'South': 47262, 'Midwest': 37136, 'West': 33069, 'Northeast': 24819})
Counter({'MSA (Metropolitan Statistical Area)': 125431, 'Non-MSA': 16855})
Counter({'Private solo or group practice': 121157, 'Other': 7819, 'Community health center': 4787, 'Freestanding clinic/urgicenter': 2444, 'Federally qualified health center (FQHC)': 2417, 'Health Maint Org (HMO)/oth prepaid prac': 2090, 'Mental health center': 750, 'Non-federal government clinic': 492, 'Faculty practice plan': 229, 'Federally qualified health ctr (FQHC)': 82, 'Family planning clinic': 19})
Counter({'Private solo or group practice': 121157, 'Other': 7819, 'Community health center': 4787, 'Freestanding clinic/urgicenter': 2444, 'Federally qualified health center (FQHC)': 2417, 'Health Maint Org (HMO)/oth prepaid prac': 2090, 'Mental health center': 750, 'Non-federal government clinic': 492, 'Faculty practice plan': 229, 'Federally qualified health ctr (FQHC)': 82, 'Family planning clinic': 19})


In [0]:
# Check if all column names are the same before concatenating dataframes

temp_df2.columns == temp_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])